# ParFuMor (version YAML & Objets)

Attention les règles qui ne changent pas le radical donnent lieu à des mauvais découpages des formes...

In [830]:
# -*- coding: utf8 -*-
from os.path import expanduser
import itertools
import yaml,YamlDuplicates
from yaml.constructor import ConstructorError
import warnings
import ParFuMor as PFM
from ParFuMor import *
import pickle
from IPython.display import HTML, display
from cellbell import ding

In [831]:
home = expanduser("~")
annee="2020"
repertoire=home+"/ownCloud/Cours/Bordeaux/L1-UE4-Morphologie/00-Kanonik/"+annee
serie=repertoire+"/"
numerosKanoniks=[1,2,3,4,5]
numerosKanoniks=[1]
nomKanoniks=[serie+"K%02d/"%num for num in numerosKanoniks]

In [832]:
def getExemple(lDict):
    exKey=lDict.keys()[0]
    if isinstance(lDict[exKey],dict):
        result=getExemple(lDict[exKey])
    elif isinstance(lDict[exKey],list):
        result=lDict[exKey][0]
    return result

In [833]:
getExemple(stems["PREP"])

'sur'

In [834]:
PFM.duplicateErrors=[]
for serie in nomKanoniks:
    print
    print serie

    with open(serie+"Gloses.yaml", 'r') as stream:
        gloses=yaml.load(stream)
        PFM.gloses=gloses
    with open(serie+"Stems.yaml", 'r') as stream:
        try:
            stems=yaml.load(stream)
            PFM.stems=stems
        except ConstructorError,msg:
            print msg
            continue
    lexiqueTestPrep=getExemple(stems["PREP"])
    lexiqueTestNoun=getExemple(stems["NOM"])
    lexiqueTestHyper=getExemple(stems["ADJ"])
    with open(serie+"Blocks.yaml", 'r') as stream:
        blocks=yaml.load(stream)
        PFM.blocks=blocks
    with open(serie+"Phonology.yaml", 'r') as stream:
        phonology=yaml.load(stream)
        PFM.phonology=phonology
    with open(serie+"MorphoSyntax.yaml", 'r') as stream:
        morphosyntax=yaml.load(stream)
        PFM.morphosyntax=morphosyntax

        
    for cat in morphosyntax["Attributs"]:
        if sorted(morphosyntax["Attributs"][cat])!=sorted(gloses[cat].keys()):
            warnings.warn("\n"+serie+"\nLes attributs de %s ne sont pas cohérents\nMorphosyntax => %s\nGloses => %s"%(cat,", ".join(morphosyntax["Attributs"][cat]),", ".join(gloses[cat].keys())))
    
    regles=Regles()
    PFM.regles=regles
    for categorie in blocks:
        regles.addBlocs(categorie,blocks[categorie])


    paradigmes=Paradigmes()
    PFM.paradigmes=paradigmes

    hierarchieCF=HierarchieCF()
    PFM.hierarchieCF=hierarchieCF
    lexique=Lexique()
    PFM.lexique=lexique


    for cat in gloses:
    #    print cat
        attributes=[]
        if gloses[cat]:
            if set(gloses[cat].keys())==set(morphosyntax["Attributs"][cat]):
                features=morphosyntax["Attributs"][cat]
    #            print "inhérent",features
            else:
                features=gloses[cat].keys()
    #            print "contextuel",features
            for attribute in features:
                attributes.append(gloses[cat][attribute])
            nuplets=(itertools.product(*attributes))
            for nuplet in nuplets:
                proprietes=[cat]
                for element in range(len(nuplet)):
                    proprietes.append(u"%s=%s"%(features[element],nuplet[element]))
                paradigmes.addForme(cat,proprietes)

    analyserGloses(gloses)
    analyserStems(stems)
        
    with open(serie+"Hierarchie-S2.pkl", 'wb') as output:
       pickle.dump(hierarchieCF, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Lexique-S2.pkl", 'wb') as output:
       pickle.dump(lexique, output, pickle.HIGHEST_PROTOCOL)
    with open(serie+"Regles-S2.pkl", 'wb') as output:
       pickle.dump(regles, output, pickle.HIGHEST_PROTOCOL)


    PFM.lexique.lexemes[lexiqueTestPrep]

    mot=lexiqueTestNoun
    classesNom=PFM.lexique.formeLexeme[mot][0].split('.')[1:]
    [classeElement for classeElement in classesNom if classeElement in gloses["NOM"]["Genre"]]

    PFM.lexique.formeLexeme[lexiqueTestHyper][0]

if PFM.duplicateErrors:
    print
    print "=======ERREURS========"
    print
    for ligne in PFM.duplicateErrors:
        print ligne



/Users/gilles/ownCloud/Cours/Bordeaux/L1-UE4-Morphologie/00-Kanonik/2020/K01/
ADJ : Genre Nombre
NOM : Genre Nombre
DET : Genre Nombre
VER : Temps Pers Genre
head stems
head stems,NOM
head stems,NOM,N1
head stems,NOM,N2


In [835]:
print PFM.lexique.formeLexeme[lexiqueTestHyper][0]
print PFM.lexique.formeLexeme[lexiqueTestNoun][0]
print PFM.lexique.formeLexeme[lexiqueTestPrep][0]

petit
femme.N1.N
dans


In [836]:
paradigmes.getSigmas(["DET"])

[u'DET, Genre=M, Nombre=Sg',
 u'DET, Genre=M, Nombre=Pl',
 u'DET, Genre=F, Nombre=Sg',
 u'DET, Genre=F, Nombre=Pl',
 u'DET, Genre=N, Nombre=Sg',
 u'DET, Genre=N, Nombre=Pl']

In [837]:
case="CF=N3"
m=re.match(ur"(.*=)(.*)","CF=N1|N2")
if m:
    altTrait=m.group(1)
    altValeurs=m.group(2).split("|")
    print altTrait
    altTraits=[altTrait+v for v in altValeurs]
    print altTraits
    if any(t == case for t in altTraits):
        print "applies"
    else:
        print "does not apply"

CF=
['CF=N1', 'CF=N2']
does not apply


In [838]:
regles.getRules("NOM",'CF=N3, Genre=C, Nombre=Du, Cas=Acc')

[]

In [839]:
test=Lexeme("RAD","DET","croc").getParadigm(lignes=["Genre"],colonne=["Nombre"])
#test=Lexeme("RAD","DET","croc").getParadigm(lignes=["Genre"],colonne=["Nombre","Cas"])
test
#test.index.sortlevel(0,ascending=False)

Nombre,Pl,Sg
Genre,,
F,RAD-i-v,RAD-i-f
M,RAD-o-v,RAD-o-f
N,RAD-e-v,RAD-e-f


In [840]:
cat="N"
triLex={}
for element in PFM.lexique.lexemes:
#    print element
    elementElts=element.split(".")
    nom=elementElts[0]
    if len(elementElts)==3:
        if elementElts[1].startswith(cat):
            if not elementElts[2] in triLex:
                triLex[elementElts[2]]=set()
            triLex[elementElts[2]].add((PFM.lexique.lexemes[element].stem,nom))            
triLex

{u'F': {('apsan', u'bruit'),
  ('di', u'jardin'),
  ('do', u'princesse'),
  ('gri', u'fille'),
  ('jole', u'Laura'),
  ('jovo', u'\xe9toile'),
  ('lisepi', u'travail'),
  ('mima', u'maman'),
  ('olo', u'bol'),
  ('roto', u'jouet'),
  ('tusi', u'lait'),
  ('zuma', u'Christelle')},
 u'M': {('afe', u'histoire'),
  ('desi', u'for\xeat'),
  ('feri', u'parent'),
  ('fre', u'b\xe9b\xe9'),
  ('fri', u'couche'),
  ('ko', u'Fran\xe7ois'),
  ('lizo', u'table'),
  ('lubo', u'papa'),
  ('nita', u'nuit'),
  ('poni', u'copain'),
  ('sefo', u'apr\xe8s-midi'),
  ('sire', u'fraise'),
  ('tulo', u'gar\xe7on'),
  ('tuplo', u'homme'),
  ('za', u'Hugo'),
  ('zolu', u'c\xe9r\xe9ale')},
 u'N': {('desa', u'cuisine'),
  ('eva', u'matin'),
  ('fiza', u'couple'),
  ('gula', u'femme'),
  ('guto', u'doudou'),
  ('nino', u'banane'),
  ('rema', u'journ\xe9e'),
  ('sibi', u'salon'),
  ('siza', u'maison'),
  ('tela', u'enfant'),
  ('ve', u'cr\xe8che'),
  ('veno', u'famille'),
  ('zalo', u'animal')}}

In [841]:
for l in PFM.lexique.lexemes:
    print l,PFM.lexique.lexemes[l].stem

manger defo
lait.N2.F tusi
Hugo.N1.M za
aimer gilo
couche.N1.M fri
aller feru
rentrer dezo
coloré ezo
petit oni
histoire.N1.M afe
rouge elu
fille.N2.F gri
vers dok
garçon.N1.M tulo
après-midi.N2.M sefo
maman.N1.F mima
jouer lopa
jouet.N2.F roto
copain.N2.M poni
DEM p
femme.N1.N gula
gros ur
céréale.N2.M zolu
papa.N2.M lubo
DEF r
homme.N2.M tuplo
jaune uve
forêt.N1.M desi
avec wij
content ak
journée.N1.N rema
calme ute
sur tor
doudou.N1.N guto
bol.N2.F olo
jardin.N1.F di
famille.N1.N veno
fraise.N2.M sire
oublier lezi
travail.N1.F lisepi
François.N2.M ko
arriver kila
dormir rumi
rester zeru
maison.N2.N siza
à fozi
beau ita
de bew
table.N2.M lizo
crèche.N1.N ve
matin.N1.N eva
nuit.N1.M nita
Laura.N1.F jole
garder seli
salon.N1.N sibi
bébé.N1.M fre
couple.N1.N fiza
Christelle.N2.F zuma
sans vik
malgré mul
prendre kolu
ensoleillé are
étoile.N2.F jovo
être zalopu
enfant.N2.N tela
bruit.N1.F apsan
dans pum
pour nuz
joyeux oka
banane.N1.N nino
animal.N2.N zalo
IND k
sucré amo
parent.N2.M feri

In [842]:
ding()